In [1]:
import re
import numpy as np
import datetime
import pandas as pd
pd.set_option('max_colwidth', None)

import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
#nltk.download('punkt')
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD

from scipy.cluster.vq import kmeans, vq

from utils import clean_data, preprocess
from utils import remove_url
from tqdm import tqdm

In [ ]:
from utils import detect_language

In [4]:
BMBF = pd.read_csv("../data/cleanedBMBF.csv")
BMBF.head(2)

,FKZ,Ressort,Referat,PT,Arb.-Einh.,Zuwendungsempfänger,Gemeindekennziffer,Stadt/Gemeinde,Ort,Bundesland,...,Staat.1,Thema,Leistungsplansystematik,Klartext Leistungsplansystematik,Laufzeit von,Laufzeit bis,Fördersumme in EUR,Förderprofil,Verbundprojekt,Förderart
0,03F0212C/7,BMBF,724,PT-J,MGS1,Universität Rostock,13003000.0,Rostock,Rostock,Mecklenburg-Vorpommern,...,Deutschland,"Verbundprojekt MESSIN: Entwicklung und Systemintegration der Komponenten Navigation, Automatische Steuerung, Kommunikation und Energieversorgung für den Meßdelphin MESSIN",FB4010,"Technik-/Infrastrukturentwicklung und -bereitstellung (Überwachungssysteme, Begleitforschung zum Monitoring, innovative Technik und Geräteträger)",01.01.1998,31.08.2000,"648.462,00",Forschung und Entwicklung zur Daseinsvorsorge,MESSIN,PDIR
1,D251800/0,BMBF,314,BIBB,A3.3,"Ministerium für Wirtschaft, Arbeit und Wohnungsbau Baden-Württemberg",8111000.0,Stuttgart,Stuttgart,Baden-Württemberg,...,Deutschland,Prozeßorientierte Entwicklungsplanung und Qualifizierung im Betrieb,OB1050,Versuchs- und Modelleinrichtungen und -programme im Bereich der beruflichen Bildung,01.07.1997,30.06.2000,"96.634,00",Forschung und Entwicklung zur Daseinsvorsorge,NaN,PDIR


In [5]:
BMBF[BMBF["Ressort"]=="BMBF"].shape

(174404, 26)

In [6]:
enrichedBMBF = pd.read_csv("../data/BMBF_idw_enrichment.csv")
enrichedBMBF.sample(3)

/var/folders/b8/g_kq7hks5k514lxq387j61600000gn/T/ipykernel_57649/1143591772.py:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  enrichedBMBF = pd.read_csv("../data/BMBF_idw_enrichment.csv")


,Unnamed: 0,FKZ,Ressort,Referat,PT,Arb.-Einh.,Zuwendungsempfänger,Gemeindekennziffer,Stadt/Gemeinde,Ort,...,Thema,Leistungsplansystematik,Klartext Leistungsplansystematik,Laufzeit von,Laufzeit bis,Fördersumme in EUR,Förderprofil,Verbundprojekt,Förderart,idw_enrichment
235672,235672,67K11695,BMU,IKIII3,PT-ZUG,NKI,Gemeinde Ebhausen,8235020.0,Ebhausen,Ebhausen,...,KSI: Umruestung der gesamten Strassenbeleuchtung im OT Rotfelden,FA1913,KSI - Klimaschutztechnologien bei der Stromnutzung,01.08.2019,31.03.2020,"9.663,00",Forschung und Entwicklung zur Daseinsvorsorge,NaN,PDIR,NaN
211130,211130,NB107B,BMBF,333,BF,NaN,Leibniz-Institut für die Pädagogik der Naturwissenschaften und Mathematik,1002000.0,Kiel,Kiel,...,Entwicklung von Forschungspotentialen im Bereich Bildung für nachhaltige Entwicklung,OB5000,Programm Zukunft Bildung,01.05.2010,31.12.2010,"15.300,00",Forschung und Entwicklung zur Daseinsvorsorge,NaN,PDIR,NaN
154845,154845,4401B00553,BMVI,DG11,ATENE,SB,Gemeinde Bennewitz,14729030.0,Bennewitz,Bennewitz,...,Beratungsleistung - Ausbau der Breitbandinfrastruktur,ZMAN,LP für alle Mandanten,27.05.2016,26.08.2017,"49.900,00",Ministerium,NaN,PDIR,NaN


In [7]:
# remove 10 NaN value in column "Thema" as indicated in the 1.notebook and column "Unnamed: 0", which is the index column of the enriched dataset  
enrichedBMBF = enrichedBMBF[enrichedBMBF["Thema"].notnull()].drop("Unnamed: 0",axis = 1)

In [8]:
# select where  there is a value in idw_enrichment
mask = ~enrichedBMBF["idw_enrichment"].isnull()

In [9]:
# merge "Thema" with "idw_enrichment" 
enrichedBMBF.loc[mask,"enriched_Thema"] = enrichedBMBF.loc[mask,"Thema"] +  enrichedBMBF.loc[mask,"idw_enrichment"]
enrichedBMBF.loc[~mask,"enriched_Thema"] = enrichedBMBF.loc[~mask,"Thema"]
enrichedBMBF.head(2)

,FKZ,Ressort,Referat,PT,Arb.-Einh.,Zuwendungsempfänger,Gemeindekennziffer,Stadt/Gemeinde,Ort,Bundesland,...,Leistungsplansystematik,Klartext Leistungsplansystematik,Laufzeit von,Laufzeit bis,Fördersumme in EUR,Förderprofil,Verbundprojekt,Förderart,idw_enrichment,enriched_Thema
0,03F0212C/7,BMBF,724,PT-J,MGS1,Universität Rostock,13003000.0,Rostock,Rostock,Mecklenburg-Vorpommern,...,FB4010,"Technik-/Infrastrukturentwicklung und -bereitstellung (Überwachungssysteme, Begleitforschung zum Monitoring, innovative Technik und Geräteträger)",01.01.1998,31.08.2000,"648.462,00",Forschung und Entwicklung zur Daseinsvorsorge,MESSIN,PDIR,NaN,"Verbundprojekt MESSIN: Entwicklung und Systemintegration der Komponenten Navigation, Automatische Steuerung, Kommunikation und Energieversorgung für den Meßdelphin MESSIN"
1,D251800 /0,BMBF,314,BIBB,A3.3,"Ministerium für Wirtschaft, Arbeit und Wohnungsbau Baden-Württemberg",8111000.0,Stuttgart,Stuttgart,Baden-Württemberg,...,OB1050,Versuchs- und Modelleinrichtungen und -programme im Bereich der beruflichen Bildung,01.07.1997,30.06.2000,"96.634,00",Forschung und Entwicklung zur Daseinsvorsorge,NaN,PDIR,NaN,Prozeßorientierte Entwicklungsplanung und Qualifizierung im Betrieb


In [10]:
enrichedBMBF.loc[~enrichedBMBF["idw_enrichment"].isna(),["idw_enrichment","enriched_Thema"]]


idw_enrichment  \
10078                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [11]:
# remove urls in enriched texts
enrichedBMBF["enriched_Thema"] = enrichedBMBF["enriched_Thema"].apply(remove_url)


In [12]:
vectorizer = CountVectorizer(tokenizer=preprocess)
svd = TruncatedSVD(n_components=400, random_state=42)

In [13]:
dbscan = DBSCAN(min_samples=799, metric="cosine")

### the following cell will perform clustering using dbscan and kmeans clustering on Ressort BMBF, after that we calculate the average silhouette scores for the each clustering method


In [14]:
%%time
topics = ["Thema","enriched_Thema"]
k = 300 
X_vec = dict()
X_SvD = dict()
dbscan_models = dict()
kmeans_models = dict()
dbscan_silhouette_avgs = dict()
kmeans_silhouette_avgs = dict()
times = dict()

for top in tqdm(topics):
    
    X = vectorizer.fit_transform(enrichedBMBF.loc[enrichedBMBF['Ressort']=='BMBF',top])
    end_time = datetime.datetime.now()
   
    X_vec[top] = X
    X_SvD[top] = svd.fit_transform(X)
    
    print("clustering using dbscan")
    start_time = datetime.datetime.now() 
    dbscan.fit(X_SvD[top])
    end_time = datetime.datetime.now()
    dbscan_models[top] = dbscan
    times[dbscan_models[top]] = end_time - start_time
    
    print("clustering using kmeans")
    start_time = datetime.datetime.now() 
    kmeans = KMeans(n_clusters=300, random_state= 42, n_init="auto").fit(X_SvD[top])
    kmeans_models[top] = kmeans
    
    end_time = datetime.datetime.now()
    times[kmeans_models[top]] = end_time - start_time
    
    
    kmeans_silhouette_avgs[top] = silhouette_score(X_SvD[top],kmeans.labels_,metric="cosine")
    print(kmeans_silhouette_avgs[top])

    dbscan_silhouette_avgs[top] = silhouette_score(X_SvD[top],dbscan.labels_,metric="cosine")
    print(dbscan_silhouette_avgs[top])
    

  0%|                                                                                             | 0/2 [00:00<?, ?it/s]/Users/isaacduong/BMBF_Clustering_Project/BMBFenv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


clustering using dbscan
clustering using kmeans
0.11368159788356888


 50%|█████████████████████████████████████████▌                                         | 1/2 [37:12<37:12, 2232.10s/it]

0.055333228747554135


/Users/isaacduong/BMBF_Clustering_Project/BMBFenv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


clustering using dbscan
clustering using kmeans
0.07603809917646455


100%|█████████████████████████████████████████████████████████████████████████████████| 2/2 [1:16:18<00:00, 2289.49s/it]

0.062391550592196476
CPU times: user 4h 15min 22s, sys: 30min 23s, total: 4h 45min 45s
Wall time: 1h 16min 19s


In [17]:
dbscan_silhouette_avgs

{'Thema': 0.055333228747554135, 'enriched_Thema': 0.062391550592196476}

In [18]:
kmeans_silhouette_avgs

{'Thema': 0.11368159788356888, 'enriched_Thema': 0.07603809917646455}

## we can see that the silhouette score of both dbscan and kmeans clustering are close to 0, which means that the documents do not show a clear cluster structure but rather randomly distributed. The data enrichment does not contribute to the clustering quality due to the very little data enrichment ( ~0,02%)

In [33]:
%%time
# IN the following we wil attempt to cluster total dataset  
svd = TruncatedSVD(n_components=400, random_state=42)

X = vectorizer.fit_transform(enrichedBMBF.loc[:,"Thema"])
X_SvD = svd.fit_transform(X)


/Users/isaacduong/BMBF_Clustering_Project/BMBFenv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CPU times: user 9min 53s, sys: 1min 15s, total: 11min 9s
Wall time: 3min 28s


In [36]:
%%time
dbscan = DBSCAN(min_samples=799, metric="cosine")
dbscan.fit(X_SvD)

CPU times: user 1h 32min 2s, sys: 11min 44s, total: 1h 43min 47s
Wall time: 24min 24s


DBSCAN(metric='cosine', min_samples=799)

In [37]:
silhouette_score(X_SvD,dbscan.labels_,metric="cosine")

0.04459172961027618

In [34]:
%%time

kmeans = KMeans(n_clusters=300, random_state= 42, n_init="auto").fit(X_SvD)

CPU times: user 6min 13s, sys: 14.1 s, total: 6min 27s
Wall time: 1min 7s


In [35]:
silhouette_score(X_SvD,kmeans.labels_,metric="cosine")

0.15477736341770243

In [ ]:
%%time
tfidf_vectorizer = TfidfVectorizer(
                                   use_idf=True, 
                                   tokenizer=preprocess)
                                 
X_tf = tfidf_vectorizer.fit_transform(BMBF.loc[BMBF['Ressort']=='BMBF','Thema'])